In [ ]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import functions as f
from pyspark import SparkContext

import pandas as pd

In [ ]:
spark = SparkSession\
            .builder\
            .master("spark://spark-master:7077")\
            .config("spark.sql.repl.eagerEval.enabled", "true")\
            .config('spark.extraListeners', 'sparkmonitor.listener.JupyterSparkMonitorListener')\
            .config('spark.driver.extraClassPath', '/opt/bitnami/python/lib/python3.6/site-packages/sparkmonitor/listener_2.12.jar')\
            .getOrCreate()

In [ ]:
pdf = pd.read_json("arxiv-metadata-oai-snapshot.json", lines=True, nrows=100)
df = spark.createDataFrame(pdf)

In [ ]:
@f.udf("integer")
def lettercount(inp: str) -> int:
    return len(inp)

@f.udf("integer")
def wordcount(inp: str) -> int:
    words = inp.split(" ")
    return len(words)

In [ ]:
df_w_lettercount = df.withColumn("lettercount", lettercount(F.col("abstract")))\
                        .withColumn("wordcount", wordcount(F.col("abstract")))

In [ ]:
df_w_lettercount.select("abstract","lettercount", "wordcount").limit(3)

In [ ]:
spark.stop()